#뉴스 선정(GoogleNews)

특정 종목의 일자별 뉴스 검색.
내용 관련성 -> 특정 키워드 기반 필터링
유사성 -> TF-IDF(흔한 단어는 가중치 낮추고 중요한 단어는 가중치 높임) 및 코사인 유사도(두 문서가 얼마나 유사한지 측정해 중복된 뉴스를 제거) 기반 분석
최대 5개 뉴스로 제한, 부족 시 추가 확보

In [2]:
%pip install GoogleNews

  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2023.3.post1
    Uninstalling pytz-2023.3.post1:
      Successfully uninstalled pytz-2023.3.post1

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:

# 1. 라이브러리 임포트

%pip install yfinance  # (Colab 등에서 필요 시)
%pip install tensorflow
%pip install matplot
import yfinance as yf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

import random


In [ ]:
#3. 데이터 수집 (주가 + 임의 감정 점수)
import yfinance as yf

ticker = 'AAPL'  # 애플 예시
start_date = '2015-01-01'
end_date = '2025-01-01'

df = yf.download(ticker, start=start_date, end=end_date)
# df: ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
df.dropna(inplace=True)
# 감정분석 값 추가 (기본값 = 0)
df['Sentiment'] = 0



In [ ]:
###################################
# 4. 멀티피처 (Close, Volume, Sentiment) + 정규화
###################################
feature_cols = ['Close', 'Volume', 'Sentiment']
data = df[feature_cols].copy()

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# DataFrame 형태로 다시 저장(편의용)
scaled_df = pd.DataFrame(scaled_data, columns=feature_cols, index=df.index)



In [ ]:
###################################
# 5. 시계열 윈도우 생성 함수
###################################
def create_sequences_multi_features(scaled_df, window_size=7, target_col='Close'):
    """
    window_size일 동안의 [Close,Volume,Sentiment]를 보고 
    다음 날(target_col)의 값을 예측하는 구조.
    """
    X_list = []
    y_list = []
    dates = scaled_df.index
    date_list = []

    target_idx = scaled_df.columns.get_loc(target_col)

    for i in range(len(scaled_df) - window_size):
        X_window = scaled_df.iloc[i : i + window_size].values
        y_value = scaled_df.iloc[i + window_size, target_idx]
        
        X_list.append(X_window)
        y_list.append(y_value)
        date_list.append(dates[i + window_size])  # 이 샘플의 '정답' 날짜(예: 예측 시점)

    X_arr = np.array(X_list)
    y_arr = np.array(y_list)
    return X_arr, y_arr, date_list

window_size = 30
X_all, y_all, date_all = create_sequences_multi_features(scaled_df, window_size=window_size, target_col='Close')

print("X_all shape:", X_all.shape)  # 예: (N, 30, 3)
print("y_all shape:", y_all.shape)  # 예: (N,)

###################################
# 6. Walk-Forward Validation 설정
###################################
def walk_forward_validation(X, y, dates, train_ratio=0.8, n_splits=3, window_size=30):
    """
    전체 데이터의 80%까지를 초기 Train으로 하고,
    이후를 n_splits 등분하여 Expanding Window 방식으로 테스트하는 예시입니다.
    각 분할에서 모델 학습 시, 에포크마다 loss가 출력됩니다.
    """
    n_total = len(X)
    n_initial_train = int(n_total * train_ratio)

    # 초반 80%를 기본 훈련 세트로 하고, 나머지 20%를 n_splits 구간으로 나눔
    X_train_initial = X[:n_initial_train]
    y_train_initial = y[:n_initial_train]
    dates_train_initial = dates[:n_initial_train]

    X_test_part = X[n_initial_train:]
    y_test_part = y[n_initial_train:]
    dates_test_part = dates[n_initial_train:]
    
    # n_splits로 나눔
    chunk_size = len(X_test_part) // n_splits

    results = []
    start_idx = 0
    
    # Expanding window 방식 (각 분할마다 누적된 데이터로 학습)
    for split_idx in range(n_splits):
        end_idx = start_idx + chunk_size
        if split_idx == n_splits - 1:  # 마지막 구간 처리
            end_idx = len(X_test_part)

        # 현재 테스트 구간
        X_test_current = X_test_part[start_idx:end_idx]
        y_test_current = y_test_part[start_idx:end_idx]
        dates_test_current = dates_test_part[start_idx:end_idx]

        # Train 데이터: 초기 Train + 지금까지의 Test 일부
        X_train_current = np.concatenate([X_train_initial, X_test_part[:start_idx]], axis=0)
        y_train_current = np.concatenate([y_train_initial, y_test_part[:start_idx]], axis=0)

        # 모델 학습 (verbose=1로 설정하여 각 에포크의 손실을 출력)
        # 수정: X_all 대신 X_train_current의 피처 수 사용
        model = build_lstm_model(input_shape=(window_size, X_train_current.shape[2]))
        print(f"\n[Split {split_idx}] Training on {len(X_train_current)} samples; Testing from {dates_test_current[0]} to {dates_test_current[-1]}")
        history = model.fit(
            X_train_current, y_train_current,
            epochs=10, batch_size=32, verbose=1
        )

        # 예측 및 RMSE 계산
        preds = model.predict(X_test_current)
        mse = mean_squared_error(y_test_current, preds)
        rmse = sqrt(mse)

        results.append({
            'split': split_idx,
            'test_range': (dates_test_current[0], dates_test_current[-1]),
            'rmse': rmse,
            'history': history  # 원한다면 각 분할의 history도 저장 가능
        })

        start_idx = end_idx  # 다음 구간으로 이동

    return results

###################################
# 7. LSTM(2레이어) 모델 구성 함수
###################################
def build_lstm_model(input_shape):
    """
    2레이어 LSTM(64, 32) + Dropout(0.2) + Dense(1)
    """
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

###################################
# 8. Walk-Forward Validation 실행
###################################
results = walk_forward_validation(X_all, y_all, date_all, train_ratio=0.8, n_splits=3)

###################################
# 9. 결과 분석
###################################
for r in results:
    print(f"Split {r['split']} | Test Range: {r['test_range'][0]} ~ {r['test_range'][1]} | RMSE: {r['rmse']:.4f}")

avg_rmse = np.mean([r['rmse'] for r in results])
print(f"Average RMSE across splits: {avg_rmse:.4f}")

###################################
# 10. 최종 모델 학습 (단순 분할)
# 전체 데이터의 80%를 Train, 나머지 20%를 Test로 단순 분할
simple_split = int(len(X_all) * 0.8)
X_train_simple, X_test_simple = X_all[:simple_split], X_all[simple_split:]
y_train_simple, y_test_simple = y_all[:simple_split], y_all[simple_split:]

model_final = build_lstm_model((window_size, X_all.shape[2]))
# validation_split=0.2를 추가하여, 학습 과정 중 검증 손실을 기록
history_final = model_final.fit(X_train_simple, y_train_simple, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

preds_simple = model_final.predict(X_test_simple)
rmse_simple = sqrt(mean_squared_error(y_test_simple, preds_simple))
print(f"[Simple final split] RMSE: {rmse_simple:.4f}")

# 학습 성능 시각화 (최종 모델)
plt.plot(history_final.history['loss'], label='Training Loss')
plt.plot(history_final.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Final Model Training/Validation Loss")
plt.show()

In [ ]:
### 실제 미래 값 예측


def predict_future(model, last_window, steps):
    """
    model: 학습된 모델 (예: model_final)
    last_window: 마지막 시계열 윈도우 데이터, shape = (window_size, feature 수)
    steps: 예측할 미래 시점의 수 (예: 5일)
    
    반환: 예측된 미래 'Close' 값들의 배열 (정규화된 값)
    """
    predictions = []
    # 현재 윈도우를 복사 (매 반복마다 업데이트됨)
    current_window = last_window.copy()
    
    for i in range(steps):
        # 현재 윈도우에 대해 모델 예측 (입력 shape: (1, window_size, feature 수))
        current_window_expanded = np.expand_dims(current_window, axis=0)
        pred_norm = model.predict(current_window_expanded)[0, 0]  # 예측된 종가 (정규화된 값)
        predictions.append(pred_norm)
        
        # 미래의 다른 피처(Volume, Sentiment)는 최근 관측치를 그대로 사용
        # 새 행: 예측된 종가 + 마지막 행의 Volume, Sentiment 값 복사
        new_row = current_window[-1].copy()
        new_row[0] = pred_norm  # 종가는 예측값으로 대체
        
        # 슬라이딩 윈도우 업데이트: 기존 윈도우의 첫 행 제거하고 새 행 추가
        current_window = np.vstack([current_window[1:], new_row])
    
    return np.array(predictions)

# 마지막 윈도우 데이터 (X_all에서 가장 마지막 샘플)
last_window = X_all[-1]  # shape: (window_size, 3)

# 예측할 미래 시점의 수 (예: 앞으로 5일)
steps = 5
future_predictions = predict_future(model_final, last_window, steps)
print("Future predicted normalized 'Close' values:", future_predictions)

# 만약 원래 스케일(예: 달러 단위)로 변환하고 싶다면,
# 예측값 배열을 [예측값, dummy, dummy] 형식으로 만들어 scaler.inverse_transform을 적용할 수 있습니다.
# 예를 들어:
dummy = np.zeros_like(future_predictions)
pred_array = np.column_stack((future_predictions, dummy, dummy))
# scaled_df에서 사용한 scaler를 적용 (단, 이 방법은 예시이며 실제로는 적절한 값 대체가 필요합니다)
preds_original = scaler.inverse_transform(pred_array)[:, 0]
print("Future predicted 'Close' values (original scale):", preds_original)

예측 + 시각화

import pandas as pd
from datetime import timedelta

# 1. 데이터의 날짜 정보를 이용해 훈련/테스트 인덱스 선택
# date_all는 create_sequences_multi_features 함수에서 반환된 날짜 리스트 (pandas DatetimeIndex 혹은 리스트)
last_date = date_all[-1]  # 전체 시퀀스의 마지막 날짜

# "2주 전"까지의 데이터를 훈련으로 사용하기 위한 종료 날짜: 
# 즉, 2주 전 이전의 시퀀스만 훈련에 포함
train_end_date = last_date - pd.Timedelta(weeks=2)

# 테스트 데이터는 "마지막 주"에 해당하는 시퀀스
test_start_date = last_date - pd.Timedelta(weeks=1)  # 마지막 주 시작일
test_end_date = last_date  # 마지막 날짜

# 훈련 인덱스: 날짜가 train_end_date보다 이전인 경우 선택
train_indices = [i for i, d in enumerate(date_all) if d < train_end_date]
# 테스트 인덱스: 날짜가 test_start_date와 test_end_date 사이인 경우 선택
test_indices = [i for i, d in enumerate(date_all) if (d >= test_start_date) and (d <= test_end_date)]

print(f"Train indices: {len(train_indices)} samples")
print(f"Test indices: {len(test_indices)} samples")

# 2. 훈련/테스트 데이터 추출
X_train_custom = X_all[train_indices]
y_train_custom = y_all[train_indices]
X_test_custom = X_all[test_indices]
y_test_custom = y_all[test_indices]

# 3. 모델 구성, 학습 및 예측 (이미 정의된 build_lstm_model 사용)
model_custom = build_lstm_model(input_shape=(window_size, X_all.shape[2]))

# 훈련: validation_split을 사용할 수도 있습니다.
history_custom = model_custom.fit(X_train_custom, y_train_custom, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# 4. 테스트 데이터에 대한 예측
preds_custom = model_custom.predict(X_test_custom)

# 5. 성능 평가: RMSE 계산 (정규화된 값 기준)
rmse_custom = sqrt(mean_squared_error(y_test_custom, preds_custom))
print(f"[Custom Split Prediction] RMSE: {rmse_custom:.4f}")

# 6. (선택) 예측값을 원래 스케일로 변환하고 싶다면:
dummy = np.zeros((preds_custom.shape[0], 2))  # Volume, Sentiment 자리에 채워 넣을 더미 값
pred_array = np.concatenate((preds_custom, dummy), axis=1)
preds_original = scaler.inverse_transform(pred_array)[:, 0]
print("Predicted 'Close' values (original scale):", preds_original)

# 테스트 데이터의 실제 y값(정규화된 값)을 원본 스케일로 변환
# 여기서 y_test_custom은 정규화된 타깃 값입니다.
dummy = np.zeros((len(y_test_custom), 2))  # 'Volume', 'Sentiment' 자리에 들어갈 더미 값 (0)
y_test_array = np.column_stack((y_test_custom, dummy))
y_test_original = scaler.inverse_transform(y_test_array)[:, 0]

# 예측된 정규화된 y값(preds_custom)을 이미 원본 스케일로 변환한 preds_original을 사용합니다.
# preds_original는 [230.95845902, 231.90000591, 232.90456854, 233.68290735, 234.17207753]로 출력됨

import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.plot(range(len(y_test_original)), y_test_original, marker='o', linestyle='-', color='blue', label='Actual Close')
plt.plot(range(len(preds_original)), preds_original, marker='o', linestyle='--', color='red', label='Predicted Close')
plt.xlabel('Test Sample (Day)')
plt.ylabel('Close Price (Original Scale)')
plt.title('Actual vs Predicted Close Prices (Last Week)')
plt.legend()
plt.show()